![title](house_prices.jpg)

In [ ]:
import pandas as pd
pd.set_option("display.max_columns", 1000)
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.impute import SimpleImputer
from sklearn.utils.validation import check_is_fitted
from itertools import product
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# Data loading and cleaning

Pandas is a standard library for data manipulation: https://pandas.pydata.org/docs/user_guide/index.html

## Data loading

In [ ]:
df = pd.read_csv("house_sales_prices.csv")

In [ ]:
# Pandas' base object is the dataframe
df.head()

In [ ]:
df.describe()

## Drop missing values

In [ ]:
df[df.isnull().any(axis=1)]

In [ ]:
# TODO: trouver et appliquer la méthode des dataframes pandas 
# permettant de retirer toutes les colonnes contenant des valeurs manquantes (None, Na)
df_with_dropped_na = ?? 
assert(len(df_with_dropped_na.columns) == 62)

In [ ]:
df_with_dropped_na.head()

## Conserver seulement les colonnes numériques

In [ ]:
df_with_dropped_na.dtypes.head(12)

In [ ]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
df_numeric = df_with_dropped_na.select_dtypes(numerics)

In [ ]:
df_numeric.dtypes

# Data visualization

seaborn est une librairie haut-niveau de visualisation de données, 
basée sur une librairie plus standard mais plus bas-niveau, matplotlib
https://seaborn.pydata.org/examples/index.html

In [ ]:
sns.histplot(df_numeric.SalePrice)

In [ ]:
correlations = df_numeric.corr()
most_correlated_features = correlations["SalePrice"].sort_values(ascending=False)[:15]
most_correlated_features

In [ ]:
correlations_most_correlated_features = df_numeric[most_correlated_features.index].corr()
sns.heatmap(correlations_most_correlated_features, cmap="coolwarm")

In [ ]:
sns.boxplot(x=df_numeric.OverallQual, y=df_numeric.SalePrice)

# Premier modèle

## Cible et variables explicatives

In [ ]:
target = "SalePrice"
y = df_numeric[target]

In [ ]:
x = df_numeric.drop(target, axis=1)
features = x.columns.tolist()
x.head()

## Séparation des données d'entraînement et de test

![title](training_test.png)

scikit-learn (importé en tant que sklearn) est la principale librairie permettant de faire du machine learning en python : https://scikit-learn.org/stable/user_guide.html

In [ ]:
test_size_ratio = 0.2
random_state = 123 
# variable à utiliser pour pouvoir reproduir la même répartition sur plusiurs exécutions

# TODO: trouver et utiliser la fonction de sklearn permettant de 
# "splitter" (séparer) des variables explicatives (x) et une cible (y)
# en un jeu d'entraînement (x_train et y_train) et un jeu de test (x_test et y_test),
# avec 80% de données d'entraînement et 20% de données de test
x_train, x_test, y_train, y_test = ??
assert(len(x_test) / (len(x_train) + len(x_test)) == test_size_ratio)

In [ ]:
x_train.shape, x_test.shape

## Entraînement du modèle

In [ ]:
tree_model = DecisionTreeRegressor(max_depth=8)

In [ ]:
# TODO: utiliser la méthode de l'objet tree_model permettant de le "fitter" (l'entraîner)
# sur les données d'entraînement
tree_model.??
check_is_fitted(tree_model)

## Prédictions sur l'ensemble d'apprentissage

In [ ]:
# TODO: utiliser la méthode de l'objet tree_model entraîné 
# permettant de générer des prédictions sur les données d'entraînement 
predictions_train = tree_model.??
assert(len(predictions_train) == len(x_train))

In [ ]:
mean_absolute_error(predictions_train, y_train)

Le problème, c'est que la valeur d'erreur ci-dessus peut prendre des valeurs très différentes en fonction de la problématique. On préférera parfois utiliser des valeurs d'erreurs qui ont peuvent prendre des valeurs plus circonscrites, comme la fonction r2 utilisée ci-dessous.

![title](r2.png)

In [ ]:
r2_score(predictions_train, y_train)

In [ ]:
predictions_vs_realite_train = pd.DataFrame({"predictions sur ensemble d'entrainement": predictions_train,
                                           "valeurs ensemble d'entrainement": y_train})
predictions_vs_realite_train.head(15)

In [ ]:
predictions_vs_realite_train.plot.scatter(x="predictions sur ensemble d'entrainement", y="valeurs ensemble d'entrainement")

## Predictions sur l'ensemble de test

In [ ]:
# TODO: utiliser la méthode de l'objet tree_model entraîné 
# permettant de générer des prédictions sur les données de test 
predictions_test = tree_model.?? 
r2_score(predictions_test, y_test)

In [ ]:
predictions_vs_realite = pd.DataFrame({"predictions sur ensemble de test": predictions_test,
                                       "valeurs ensemble de test": y_test})
predictions_vs_realite.plot.scatter(x="predictions sur ensemble de test", y="valeurs ensemble de test")

# Recherche des meilleurs paramètres

![titile](training_and_test.png)

In [ ]:
# Création de l'ensemble de validation et d'un nouveau ensemble d'entraînement et 
# à partir de l'ancien ensemble d'entraînement
x_training, x_val, y_training, y_val = train_test_split(x_train, y_train)

In [ ]:
hyperparameters_grid = {"max_depth": [None] + list(range(2, 12)), 
                        "min_samples_split": list(range(2, 20))}

# Créons la liste de toutes les combinaisons possibles d'hyperparamètres
hyperparameters_combinations_tuple_list = product(*(hyperparameters_grid[key] 
                                                    for key in hyperparameters_grid))
hyperparameters_combinations_dict_list = [{"max_depth": l[0], 
                                           "min_samples_split": l[1]} for l in
                                          hyperparameters_combinations_tuple_list]
hyperparameters_combinations_dict_list

In [ ]:
def get_score_with_a_decison_tree(hyperparameters):
    tree = DecisionTreeRegressor(**hyperparameters)
    # TODO: utiliser les méthodes vue précédemment
    # pour entraîner le modèle tree sur le nouvel ensemble d'entraînement 
    # puis retourner son score R2 sur l'ensemble de validation
    ??
    predictions = ??
    score = ??
    return score

assert(get_score_with_a_decison_tree({'max_depth': None, 'min_samples_split': 2}) <= 1)

In [ ]:
# Pour chaque combinaison d'hyperparamètres, entraînons un arbre
# et calculons son score sur l'ensemble de validation
scores = [get_score_with_a_decison_tree(hyperparameter_combination) 
          for hyperparameter_combination in hyperparameters_combinations_dict_list]

In [ ]:
max_score = max(scores)
print("Score du meilleur modèle sur l'ensemble où les hyperparamètres ont été optimisés: %s" % max_score)
best_score_index = scores.index(max_score)
best_hyperparameters = hyperparameters_combinations_dict_list[best_score_index]
best_tree = DecisionTreeRegressor(**best_hyperparameters).fit(x_train, y_train)
print("Score du meilleur modèle sur l'ensemble de test: %s" % best_tree.score(x_test, y_test))

print("Meilleurs hyperparamètres: %s" % best_hyperparameters)

In [ ]:
predictions_best_tree_vs_realite = pd.DataFrame({"predictions sur ensemble de test": best_tree.predict(x_test),
                                                 "valeurs ensemble de test": y_test})
predictions_best_tree_vs_realite.plot.scatter(x="predictions sur ensemble de test", y="valeurs ensemble de test")

# Validation croisée

![title](kfolds.jpg)

In [ ]:
def get_cross_val_score(hyperparameters):
    scores = []
    x_train_matrix = x_train.values
    y_train_matrix = y_train.values
    # Créons six sous-ensembles (folds) de taille égale
    kfold = KFold(n_splits=6)
    # Et récupérons tous les ensembles d'entraînement et de validation possibles
    for train_indices, val_indices in kfold.split(x_train_matrix):
        x_train_k = x_train_matrix[train_indices, :]
        y_train_k = y_train_matrix[train_indices]
        x_val_k = x_train_matrix[val_indices, :]
        y_val_k = y_train_matrix[val_indices]
        tree = DecisionTreeRegressor(**hyperparameters)
        # TODO: entraîner le modèle sur le sous-ensemble d'entraînement,
        # et récupérer son score sur le sous-ensemble de validation
        score = ??
        scores.append(score)

    return np.mean(scores)

assert(get_cross_val_score({'max_depth': None, 'min_samples_split': 2}) <= 1)

In [ ]:
get_cross_val_score(best_hyperparameters)

In [ ]:
# Recalculons les scores de chacune des combinaisons de paramètres
cv_scores = [get_cross_val_score(hyperparameter_combination)
             for hyperparameter_combination in hyperparameters_combinations_dict_list]

In [ ]:
max_score_cv = max(cv_scores)
print("Score du meilleur modèle sur l'ensemble de validation: %s" % max_score_cv)
best_score_index_cv = cv_scores.index(max_score_cv)
best_hyperparameters_cv = hyperparameters_combinations_dict_list[best_score_index_cv]
best_tree_cv = DecisionTreeRegressor(**best_hyperparameters_cv).fit(x_train, y_train)
print("Score du meilleur modèle sur l'ensemble de test: %s" % best_tree_cv.score(x_test, y_test))

print("Meilleurs paramètres: %s" % best_hyperparameters_cv)

In [ ]:
predictions_vs_realite_cv = pd.DataFrame({"predictions sur ensemble de test": best_tree.predict(x_test),
                                       "valeurs ensemble de test": y_test})
predictions_best_tree_vs_realite.plot.scatter(x="predictions sur ensemble de test", y="valeurs ensemble de test")

# Compromis biais variance

![title](bootstrap.png)

In [ ]:
n_samples = 200
sample_size = 1000
pool_size = x_train.shape[0]

def get_bootstrap_sample(pool_size=pool_size, sample_size=sample_size):
    return np.random.choice(range(pool_size), size=sample_size, replace=True)

# Créons une liste d'indices d'échantillons "bootstrap" sur l'ensemble d'entraînement
samples = [get_bootstrap_sample() for _ in range(n_samples)]

In [ ]:
def train_individual_tree(sample, max_depth=2):
    x_train_sample = x_train.values[sample, :]
    y_train_sample = y_train.values[sample]
    tree_sample = DecisionTreeRegressor(max_depth=max_depth)
    return tree_sample.fit(x_train_sample, y_train_sample)

mean_bias, mean_variance = [], []
scores_one_tree = []
max_depths = range(1, 25, 4)
# Pour différentes valeurs de profondeur possibles...
for depth in max_depths:
    # On va entraîner des arbres de décision, un par échantillon bootstrap
    tree_samples = [train_individual_tree(sample, depth) for sample in samples]
    # TODO: créeer une liste contenant les prédictions de chacun des arbres sur x_test
    predictions_tree_samples = [?? for tree in tree_samples] 
    
    # On calcule les taux d'erreur de chacun des arbres...
    error_rates = np.concatenate([((x - y_test) / y_test).values.reshape(len(y_test), 1) 
                                for x in predictions_tree_samples],
                              axis=1)
    # Et on en déduit un taux d'erreur moyen, ou bias
    mean_bias.append(np.mean(np.abs(np.mean(error_rates, axis=1))))
    # et la variance des erreurs
    mean_variance.append(np.mean(np.std(error_rates, axis=1)))
    
    one_tree = DecisionTreeRegressor(max_depth=depth).fit(x_train, y_train)
    scores_one_tree.append(one_tree.score(x_test, y_test))

In [ ]:
plt.plot(max_depths, mean_bias)
plt.title(u"Évolution du biais en fonction de la profondeur des arbres")

In [ ]:
plt.plot(max_depths, mean_variance)
plt.title(u"Évolution de la variance en fonction de la profondeur des arbres")

In [ ]:
plt.plot(max_depths, scores_one_tree)
plt.title(u"Évolution du score d'un seul arbre en fonction de la profondeur")

![title](bias_variance.png)

# Aggrégation bootstrap (bagging)

In [ ]:
score_bootstrap_aggregation_predictions = []
# Pour chaque profondeur possible...
for depth in max_depths:
    # On va maintenant prédire la moyenne des prédictions des arbres
    # entraînés sur les échantillons bootstrap
    tree_samples = [train_individual_tree(sample, depth) for sample in samples]
    predictions_tree_samples = [tree.predict(x_test) for tree in tree_samples]
    bootstrap_aggregation_predictions = sum(predictions_tree_samples) / n_samples
    score_bootstrap_aggregation_predictions.append(
        r2_score(y_test, bootstrap_aggregation_predictions))

In [ ]:
plt.plot(max_depths, score_bootstrap_aggregation_predictions)
plt.title(u"Évolution de l'erreur de l'aggrégation bootstrap en fonction de la profondeur")

In [ ]:
# Visualisons les prédictions pour des arbres de profondeur 10
tree_samples = [train_individual_tree(sample, 10) for sample in samples]
predictions_tree_samples = [tree.predict(x_test) for tree in tree_samples]
bootstrap_aggregation_predictions = sum(predictions_tree_samples) / n_samples

predictions_vs_realite_bootstrap_aggregation = pd.DataFrame({"predictions sur ensemble de test": bootstrap_aggregation_predictions,
                                       "valeurs ensemble de test": y_test})
predictions_vs_realite_bootstrap_aggregation.plot.scatter(x="predictions sur ensemble de test", y="valeurs ensemble de test")

# Forêt d'arbres aléatoires

![title](random_forest.png)

In [ ]:
rf = RandomForestRegressor(max_depth=50, n_estimators=1000, n_jobs=-1)

In [ ]:
rf.fit(x_train, y_train)

In [ ]:
rf.score(x_test, y_test)

# Boosting

![title](boosting_trees.png)

In [ ]:
gbm = GradientBoostingRegressor(n_estimators=100, criterion="mse")
gbm.fit(x_train, y_train)

In [ ]:
gbm.score(x_test, y_test)

# Feature engineering

## Imputation des valeurs manquantes

TODO: récupérer les données de départ, et remplacer les valeurs manquantes par la moyenne ou la médianne des valeurs de la colonne, en utilisant sklearn

## Dummification

TODO: Remplacer les colonnes contenant des variables catégorielles par des colonnes contenant des 0 et des 1, indicant si l'échantillon appartient ou non à la catégorie, en utilisant sklearn